In [76]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
import os
import glob
import numpy as np
import pickle
import scipy.signal as signal
from scipy.stats import zscore
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.ensemble import HistGradientBoostingClassifier
from sklearn.metrics import classification_report, accuracy_score
from feature_cleaning import extract_features_from_signal
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
import joblib

In [78]:
def load_data(data_dir='test_data/'):
    """
    Load all .npy files from subdirectories (gesture labels) under data_dir.
    Assumes structure: data_dir/<gesture>/<gesture>_rep*.npy
    Returns:
        signals: list of raw signal arrays
        labels: list of corresponding labels (str)
    """
    signals = []
    labels = []
    for gesture_dir in os.listdir(data_dir):
        full_dir = os.path.join(data_dir, gesture_dir)
        if os.path.isdir(full_dir):
            for file in glob.glob(os.path.join(full_dir, '*.npy')):
                data = np.load(file).T
                if data.shape[0] != 7:
                    print(f"Skipping {file} with shape {data.shape}")
                    continue
                print(data.shape)
                new_data = np.concatenate((data[5],data[6]))
        
            
                signals.append(new_data)  # data is assumed to be a 2D array: samples x channels
                labels.append(gesture_dir)


    return signals, labels

In [79]:
signals, labels = load_data()

(7, 2300)
(7, 2500)
(7, 2300)


In [80]:
signal_chunck_size = 333
new_signals = []
new_labels = []
for label, signal in zip(labels, signals):

    signal1 = signal[:len(signal)//2]
    signal2 = signal[len(signal)//2:]

    for i in range(0, len(signal1), signal_chunck_size):
        if i + signal_chunck_size > len(signal1):
            i = len(signal1) - signal_chunck_size
        new_signal = np.concatenate((signal1[i:i+signal_chunck_size] ,signal2[i:i+signal_chunck_size]))

        features = extract_features_from_signal(new_signal)

        new_signals.append(features)
        new_labels.append(label)

In [81]:
new_signals[0], new_labels[-1]

(array([   2.41687722,    1.84672742,   97.        ,  183.        ,
         539.52561557,   35.15625   ,    8.80511845,    6.75916728,
          78.        ,  135.        , 1693.33358711,   66.40625   ]),
 'repos')

In [82]:
df = pd.DataFrame(new_signals, columns=["rms1", "mav1", "zc1", "ssc1", "wl1", "dom_freq1","rms2", "mav2", "zc2", "ssc2", "wl2", "dom_freq2"])
df['label'] = new_labels 

label_mapping = {'open': "open", 'punch': "punch", 'repos': "rest"}
df['label'] = df['label'].map(label_mapping)

In [83]:
df.head()

,rms1,mav1,zc1,ssc1,wl1,dom_freq1,rms2,mav2,zc2,ssc2,wl2,dom_freq2,label
0,2.416877,1.846727,97.0,183.0,539.525616,35.15625,8.805118,6.759167,78.0,135.0,1693.333587,66.40625,open
1,2.483103,1.936531,101.0,165.0,549.721653,54.68750,9.354299,6.899954,65.0,140.0,1505.108207,50.78125,open
2,1.914283,1.533502,91.0,165.0,468.707393,50.78125,9.983520,6.660797,70.0,136.0,1503.480491,50.78125,open
3,2.193632,1.713367,100.0,176.0,509.311554,54.68750,9.768186,7.060970,72.0,130.0,1482.198886,54.68750,open
4,1.922283,1.530956,108.0,177.0,489.068857,66.40625,8.223347,5.653749,74.0,138.0,1273.181659,66.40625,open


In [84]:
scaler = joblib.load("scaler.pkl")

In [85]:
# Scaling data
num_cols = df.select_dtypes(include=['number']).columns
df[num_cols] = scaler.fit_transform(df[num_cols])

In [86]:
df.head()

,rms1,mav1,zc1,ssc1,wl1,dom_freq1,rms2,mav2,zc2,ssc2,wl2,dom_freq2,label
0,0.940100,1.020945,-0.980656,0.243020,0.929446,-0.841422,1.312042,1.508556,-0.636234,-0.928307,1.995340,-0.482639,open
1,1.080440,1.294384,-0.755336,-1.334619,1.115988,-0.341117,1.516016,1.580511,-1.147366,-0.713783,1.463235,-0.799596,open
2,-0.124941,0.067224,-1.318637,-1.334619,-0.366211,-0.441178,1.749718,1.458279,-0.950776,-0.885402,1.458634,-0.799596,open
3,0.467024,0.614883,-0.811666,-0.370506,0.376664,-0.341117,1.669740,1.662805,-0.872141,-1.142832,1.398471,-0.720357,open
4,-0.107988,0.059473,-0.361025,-0.282860,0.006313,-0.040934,1.095963,0.943585,-0.793505,-0.799593,0.807589,-0.482639,open


# Test Models

In [87]:
with open('models/movement_classifier_model.pkl', 'rb') as file:
    movement_classifier = pickle.load(file)
with open('models/open_classifier_model.pkl', 'rb') as file:
    open_tired_classifier = pickle.load(file)
with open('models/punch_classifier_model.pkl', 'rb') as file:
    punch_tired_classifier = pickle.load(file)

## Movement Model

In [88]:
X_test = scaler.transform(df.drop('label', axis=1))
y_test = df['label']

y_test_pred = movement_classifier.predict(X_test)

test_accuracy = accuracy_score(y_test, y_test_pred)

print(f"Test Accuracy: {test_accuracy * 100:.2f}%")

Test Accuracy: 31.82%


c:\Users\walid\miniconda3\envs\amal\lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but HistGradientBoostingClassifier was fitted with feature names
  warnings.warn(


In [89]:
y_test_pred

array(['punch', 'punch', 'punch', 'punch', 'punch', 'punch', 'punch',
       'open', 'punch', 'punch', 'punch', 'punch', 'punch', 'punch',
       'punch', 'punch', 'punch', 'punch', 'punch', 'punch', 'punch',
       'punch'], dtype=object)